<center> <img src="https://miro.medium.com/v2/resize:fit:1200/1*lbDXL0IuitCRz4mpZ7MmfQ.png" width=55% > </center>

<br><br>

<center>
    <font size="6">Final Lab (Part 2): Image Classification using Convolutional Neural Networks </font>
</center>
<center>
    <font size="4">Computer Vision 1 University of Amsterdam</font>
</center>
<center>
    <font size="4">Due 23:59PM, October 18, 2024 (Amsterdam time)</font>
</center>
<center>
    <font size="4"><b>TA's:  Yue, Konrad & Thies</b></font>
</center>

<br><br>

***

<br><br>

<center>

Student1 ID:  \
Student1 Name:

Student2 ID: \
Student2 Name:

Student3 ID: 13853163\
Student3 Name Benjamin Shaffrey:

Student4 ID: 15050025\
Student4 Name: Christina Isaicu

</center>

### **Coding Guidelines**

Your code must be handed in this Jupyter notebook, renamed to **StudentID1_StudentID2_StudentID3.ipynb** before the deadline by submitting it to the Canvas Final Lab: Image Classification Assignment. Please also fill out your names and IDs above.

For full credit, make sure your notebook follows these guidelines:

- Please express your thoughts **concisely**. The number of words does not necessarily correlate with how well you understand the concepts.
- Understand the problem as much as you can. When answering a question, provide evidence (qualitative and/or quantitative results, references to papers, figures, etc.) to support your arguments. Not everything might be explicitly asked for, so think about what might strengthen your arguments to make the notebook self-contained and complete.
- Tables and figures must be accompanied by a **brief** description. Add a number, a title, and, if applicable, the name and unit of variables in a table, and name and unit of axes and legends in a figure.

**Late submissions are not allowed.** Assignments submitted after the strict deadline will not be graded. In case of submission conflicts, TAs’ system clock is taken as reference. We strongly recommend submitting well in advance to avoid last-minute system failure issues.

**Environment:** Since this is a project-based assignment, you are free to use any feature descriptor and machine learning tools (e.g., K-means, SVM). You should use Python for your implementation. You are free to use any Python library for this assignment, but make sure to provide a conda environment file!

**Plagiarism Note:** Keep in mind that plagiarism (submitted materials which are not your work) is a serious offense and any misconduct will be addressed according to university regulations. This includes using generative tools such as ChatGPT.

**Ensure that you save all results/answers to the questions (even if you reuse some code).**

### **Report Preparation**

Your tasks include the following:

1. **Report Preparation:** For both parts of the final project, students are expected to prepare a report. The report should include all details on implementation approaches, analysis of results for different settings, and visualizations illustrating experiments and performance of your implementation. Grading will be based on the report, so it should be as self-contained as possible. If the report contains faulty results or ambiguities, TAs can refer to your code for clarification.

2. **Explanation of Results:** Do not just provide numbers without explanation. Discuss different settings to show your understanding of the material and processes involved.

3. **Quantitative Evaluation:** For quantitative evaluation, you are expected to provide the results based on performance (accuracy, learning loss and learning curves).

4. **Aim:** Understand the basic Image Classification pipeline using Convolutional Neural Nets (CNN's).

5. **Working on Assignments:** Students should work in assigned groups for **two** weeks. Any questions can be discussed on ED.

    - **Submission:** Submit your source code and report together in a zip file (`ID1_ID2_ID3_part2.zip`). The report should be a maximum of 10 pages (single-column, including tables and figures, excluding references and appendix). Express thoughts concisely. Tables and figures must be accompanied by a description. Number them and, if applicable, name variables in tables, and label axes in figures.

6. **Hyperparameter Search:** In your experiments, remember to perform a hyperparameter search to find the optimal settings for your model(s). Clearly document the search process, the parameters you explored, and how they influenced the performance of your model.

8. **Format and Testing:** The report should be in **PDF format**, and the code in **.ipynb format**. Test that all functionality works as expected in the notebook.

### **Overview**

- [Section 1: Image Classification on CIFAR-100 (0 points)](#section-1)
- [Section 2: Visualizing CIFAR-100 Classes and Subclasses (3 points)](#section-2)
- [Section 3: TwoLayerNet Architecture (2 points)](#section-3)
- [Section 4: ConvNet Architecture (2 points)](#section-4)
- [Section 5: Preparation of Training (7 points)](#section-5)
- [Section 6: Training the Networks (5 points)](#section-6)
- [Section 7: Setting Up the Hyperparameters (14 points)](#section-7)
- [Section 8: Visualizing the STL-10 Dataset and Preparing the Data Loader (3 points)](#section-8)
- [Section 9: Fine-tuning ConvNet on STL-10 (14 points)](#section-9)
- [Section 10: Bonus Challenge (optional)](#section-10)
- [Section X: Individual Contribution Report (Mandatory)](#section-x)

### **Section 1: Image Classification on CIFAR-100 (0 points)**

The goal of this lab is to implement an image classification system using Convolutional Neural Networks (CNNs) that can identify objects from a set of classes in the [CIFAR-100 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). You will implement and compare two different architectures: a simple two-layer network and a ConvNet based on the LeNet architecture.

The CIFAR-100 dataset contains 32x32 pixel RGB images, categorized into 100 different classes. The dataset will be automatically downloaded and loaded using the code provided in this notebook.

You will train and test your classification system using the entire CIFAR-100 dataset. Ensure that the test images are excluded from training to maintain a fair evaluation of the model's performance.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from PIL import Image
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm.notebook import tqdm
from sklearn import metrics

import json
import os
import random

In [ ]:
def set_seed(seed):
    '''
    Set the seed for reproducibility.

    Args:
        seed (int): Seed value.
    '''
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
# Define the transformations

transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Load the CIFAR-100 training set
train_set = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Load the CIFAR-100 test set
test_set = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Create data loaders for the entire CIFAR-100 dataset
train_data_loader = DataLoader(train_set, shuffle=True, num_workers=8)
test_data_loader = DataLoader(test_set, shuffle=False, num_workers=8)

print("Data loaders for CIFAR-100 are ready for use.")



In [ ]:
# Define CIFAR-100 superclasses and their subclasses
superclasses = {
    'aquatic mammals': ['beaver', 'dolphin', 'otter', 'seal', 'whale'],
    'fish': ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
    'flowers': ['orchid', 'poppy', 'rose', 'sunflower', 'tulip'],
    'food containers': ['bottle', 'bowl', 'can', 'cup', 'plate'],
    'fruit and vegetables': ['apple', 'mushroom', 'orange', 'pear', 'sweet_pepper'],
    'household electrical devices': ['clock', 'keyboard', 'lamp', 'telephone', 'television'],
    'household furniture': ['bed', 'chair', 'couch', 'table', 'wardrobe'],
    'insects': ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
    'large carnivores': ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
    'large man-made outdoor things': ['bridge', 'castle', 'house', 'road', 'skyscraper'],
    'large natural outdoor scenes': ['cloud', 'forest', 'mountain', 'plain', 'sea'],
    'large omnivores and herbivores': ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo'],
    'medium-sized mammals': ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
    'non-insect invertebrates': ['crab', 'lobster', 'snail', 'spider', 'worm'],
    'people': ['baby', 'boy', 'girl', 'man', 'woman'],
    'reptiles': ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
    'small mammals': ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
    'trees': ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree'],
    'vehicles 1': ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
    'vehicles 2': ['lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor']
}

# List of all CIFAR-100 classes
classes = ('apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
           'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
           'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur',
           'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard',
           'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
           'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
           'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea',
           'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower',
           'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip',
           'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm')

# Create a mapping of class names to their indices
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

# Create a mapping of superclasses to their corresponding class indices
superclass_to_indices = {supcls: [class_to_idx[cls] for cls in subclasses] for supcls, subclasses in superclasses.items()}

<a id="section-2"></a>
### **Section 2: Visualizing CIFAR-100 Classes and Subclasses (3 points)**

In this section, you will implement a function to visualize the CIFAR-100 dataset, including **all** superclasses and their corresponding subclasses. Your implementation should provide a clear and organized overview of the dataset's diversity.

You add the figure(s) to appendix of your report and refer to it in the main text.

In [ ]:
# YOUR CODE HERE
def visualise_cifar100(train_set, superclasses, superclass_to_indices, classes):
    data = torch.tensor(train_set.data)
    labels = torch.tensor(train_set.targets)

    num_classes_per_superclass = len(superclasses['trees'])
    num_samples = num_classes_per_superclass

    for superclass in tqdm(superclasses.keys()):
        fig, axs = plt.subplots(num_classes_per_superclass, num_samples, figsize=(25, 20))

        # Set superclass name as plot title
        fig.suptitle(f'Superclass: {superclass}', fontsize=20)
        for num_class, index in enumerate(superclass_to_indices[superclass]):
            subset = data[labels == index]
            random_indices = torch.randperm(subset.size(0))
        
            # Set the class name as the label for the first column
            class_name = classes[index]
            axs[num_class, 0].set_ylabel(class_name, fontsize=16, rotation=0, labelpad=80, va='center')
        
            for num_sample, sample_index in enumerate(random_indices[ : num_samples]):
                axs[num_class, num_sample].imshow(subset[sample_index])
                axs[num_class, num_sample].set_xticks([])
                axs[num_class, num_sample].set_yticks([])

visualise_cifar100(train_set, superclasses, superclass_to_indices, classes)

<a id="section-3"></a>
### **Section 3: TwoLayerNet Architecture (2 points)**

In this section, you will implement the architecture of a fully connected neural network called `TwoLayerNet`, consisting of two fully connected layers with a ReLU activation in between. The network accepts an input size of 3x32x32 (CIFAR-100 image), a specified hidden layer size, and the number of output classes. In the `__init__` method, define the first fully connected layer that maps the input size to the hidden size, and the second fully connected layer that maps the hidden size to the number of classes.

Ensure to call the parent class constructor using `super(TwoLayerNet, self).__init__()`. In the `forward` method, flatten the input tensor, pass it through the first layer with ReLU activation, and then through the second layer to obtain the final scores.

**Note:** You are allowed to modify the provided function definitions as needed.

In [ ]:
class TwoLayerNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes, extra_layers=False):
        '''
        Initializes the two-layer neural network model.

        Args:
            input_size (int): The size of the input features.
            hidden_size (int): The size of the hidden layer.
            num_classes (int): The number of classes in the dataset.
        '''

        # YOUR CODE HERE
        super().__init__()

        if extra_layers is False:
            layers = [nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, num_classes)]

        # MLP with 2 additional linear layers & a batch norm layer
        else:
            layers = [nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, hidden_size), nn.ReLU(), nn.BatchNorm1d(hidden_size), nn.ReLU(), nn.Linear(hidden_size, num_classes)]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        '''
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor.
        '''

        # YOUR CODE HERE
        # flatten input
        x = x.flatten(start_dim=1)
        return self.model(x)


<a id="section-4"></a>
### **Section 4: ConvNet Architecture (2 points)**

In this section, you will implement a convolutional neural network inspired by the structure of [LeNet-5](https://ieeexplore.ieee.org/document/726791). The network processes color images using three convolutional layers followed by two fully connected layers. Since you need to feed color images into this network, determine the kernel size of the first convolutional layer. Additionally, calculate the number of trainable parameters in the "F6" layer, providing the calculation process.

In [ ]:
class ConvNet(nn.Module):

    def __init__(self, non_linear=None, extra_layers=False, num_classes=100):
        '''
        Initializes the convolutional neural network model.
        Follows the architecture of LeNet-5.

        Args:
            None
        '''

        # YOUR CODE HERE
        super().__init__()

        # original activation functions from LeNet-5 paper
        if non_linear is None:
            non_linear1 = nn.Tanh()
            non_linear2 = nn.Sigmoid()

        # replace all activation functions with ReLU
        elif non_linear == "relu":
            non_linear1 = nn.ReLU()
            non_linear2 = nn.ReLU()
        else:
            raise ValueError("Invalid non-linear function. Please choose 'relu' or None.")

        layers = []

        if extra_layers is False:
          # original architecture from LeNet-5

          # Layer C1 is a convolutional layer with six feature maps.
          # Each unit in each feature map is connected to a 5X5 neighborhood in the input.
          # output: 6 feature maps of size 28X28
          layers.append(nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5))
          # each layer is passed through a squashing function (hyperbolic tangent)
          layers.append(non_linear1)

          # Layer S2 is a subsampling layer with six feature maps of size 14X14.
          # Each unit in each feature map is connected to a 2X2 neighborhood in the corresponding feature map in C1.
          # output: 6 feature maps of size 14X14
          layers.append(nn.AvgPool2d(kernel_size=2, stride=2))

          # The result is passed through a sigmoidal function.
          layers.append(non_linear2)

          # Layer C3 is a convolutional layer with 16 feature maps.
          # Each unit in each feature map is connected to several 5X5
          # neighborhoods at identical locations in a subset of S2’s feature maps
          # output: 16 feature maps of size 10X10
          layers.append(nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5))
          layers.append(non_linear1)

          # Layer S4 is a subsampling layer with 16 feature maps of size 5X5.
          # Each unit in each feature map is connected to a 2X2 neighborhood
          # in the corresponding feature map in C3
          layers.append(nn.AvgPool2d(kernel_size=2, stride=2))

          # Layer C5 is a convolutional layer with 120 feature maps.
          # Each unit is connected to a 5x5 neighborhood on all 16
          # of S4’s feature maps.
          layers.append(nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5))
          layers.append(non_linear1)

          # Flatten the output of C5 (linear layer expects a 2D input of size (batch_size, num_features))
          layers.append(nn.Flatten())

          # Layer F6 contains 128 units and is fully connected to C5
          # Layer F6 has 10 164 trainable parameters
          # Total Parameters = input features ×output features + bias (of size output features)
          # 120 * 84 + 84 = 10164
          layers.append(nn.Linear(in_features=120, out_features=84))
          layers.append(non_linear1)

          layers.append(nn.Linear(in_features=84, out_features=num_classes))

        else:
          # additional batch norm, max pool, and dropouts applied (not counted as extra layers)
          # convolutional channel sizes adjusted
          layers.append(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5))
          layers.append(nn.BatchNorm2d(64))
          layers.append(nn.ReLU())
          layers.append(nn.MaxPool2d(kernel_size=2))

          layers.append(nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3))
          layers.append(nn.BatchNorm2d(96))
          layers.append(nn.ReLU())
          layers.append(nn.MaxPool2d(kernel_size=2))

          layers.append(nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3))
          layers.append(nn.BatchNorm2d(128))
          layers.append(nn.ReLU())

          # 1st addtional layer - convolutional layer
          layers.append(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3))
          layers.append(nn.BatchNorm2d(128))
          layers.append(nn.ReLU())

          layers.append(nn.Flatten(start_dim=1))

          # 2nd addtional layer - linear, in_features reshaped to match flatten
          layers.append(nn.Linear(in_features=512, out_features=256))
          layers.append(nn.ReLU())

          layers.append(nn.Dropout(p=0.55))

          layers.append(nn.Linear(in_features=256, out_features=128))
          layers.append(nn.ReLU())

          layers.append(nn.Dropout(p=0.55))

          layers.append(nn.Linear(in_features=128, out_features=num_classes))

        self.model = nn.Sequential(*layers)


    def forward(self, x):
        '''
        Defines the forward pass of the neural network.

        Args:
            x (torch.Tensor): The input tensor.

        Returns:
            torch.Tensor: The output tensor.
        '''

        # YOUR CODE HERE
        out = self.model(x)
        return out


<a id="section-5"></a>
### **Section 5: Preparation of Training (7 points)**

In this section, you will create a custom dataset class to load the CIFAR-100 data, define a transform function for data augmentation, and set up an optimizer for training. While the previous section utilized the built-in CIFAR-100 class from `torchvision`, in practice, you often need to prepare datasets manually. Here, you will implement the `CIFAR100_loader` class to handle the dataset and use `DataLoader` to make it iterable. You will also define a transform function for data augmentation and an optimizer for updating the model's parameters.

In [ ]:
class CIFAR100_loader(Dataset):

    def __init__(self, root, train=True, transform=None, download=False):
        '''
        Initializes the CIFAR-100 dataset loader.

        Args:
            root (str): The root directory to store the dataset.
            train (bool): If True, loads the training data; otherwise, loads the test data.
            transform (callable, optional): The data transformations to apply.
            download (bool): If True, downloads the dataset if it is not already available.
        '''

        # YOUR CODE HERE
        self.data = torchvision.datasets.CIFAR100(root=root, train=train, download=download, transform=transform)

    def __len__(self):
        '''
        Returns the number of samples in the dataset.

        Returns:
            int: The number of samples in the dataset.
        '''

        # YOUR CODE HERE
        return len(self.data)

    def __getitem__(self, idx):
        '''
        Retrieves a sample from the dataset at the specified index.

        Args:
            idx (int): The index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the image and label tensors.
        '''

        # YOUR CODE HERE
        return self.data[idx]

In [ ]:
# from pathlib import Path
# import pickle
# import requests
# import tarfile
# import os

# class CIFAR100_loader(Dataset):
    
#     def __init__(self, root, train=True, transform=None, download=False, validate=False):
#         self.root = root
#         self.train = train
#         self.transform = transform

#         CWD = Path(os.getcwd())
#         ROOT = CWD/Path(self.root)
#         os.makedirs(ROOT, exist_ok=True)
        
#         if download:
#             os.chdir(ROOT)
#             url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
#             file = "cifar-100-python.tar.gz"
#             req = requests.get(url, stream=True)
#             open(file, "wb").write(req.content)
#             tarfile.open(file, "r:gz").extractall(filter="fully_trusted")
#             os.chdir(CWD)

#         with open(ROOT/Path("cifar-100-python/meta"), "rb") as f:
#             meta = pickle.load(f)
#             self.all_labels = tuple(meta["fine_label_names"])

#         with open(f"./data/cifar-100-python/{"train" if train else "test"}", "rb") as f:
#             dataset = pickle.load(f, encoding="bytes")
#             self.labels = dataset[b'fine_labels']
#             self.images = []
#             for i in dataset[b'data']:
#                 img = np.zeros((32,32,3))
#                 for j in range(3):
#                     img[:,:,j] = np.array(i[j*1024 : (j+1)*1024]).reshape((32,32))
#                 self.images.append(Image.fromarray(np.uint8(img)))

#         if validate:
#             self.images = self.images[:6969]
#             self.labels = self.labels[:6969]


#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):

#         if self.transform:
#             return self.transform(self.images[idx]), self.labels[idx]
#         else:
#             to_tensor = transforms.ToTensor()
#             return to_tensor(self.images[idx]), self.labels[idx]

In [ ]:
def create_transforms(additional_transform=False):
    '''
    Creates the data transformations for the CIFAR-100 dataset.
    Args:
        additional_transform (bool): If True, apply additional transformations.

    Returns:
        torchvision.transforms.Compose: The data transformations for the dataset.
    '''

    if additional_transform is False:
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    # additional transformations for hyperparameter tuning
    elif additional_transform is True:
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    else:
        raise ValueError("Invalid transformation type. Please choose 'original' or 'augmented'.")

    return transform



In [ ]:
def create_optimizer(model, learning_rate=0.001, weight_decay=1.0, optimizer='adam'):
    '''
    Creates an optimizer for the model.

    Args:
        model (torch.nn.Module): The neural network model.
        learning_rate (float): The learning rate for the optimizer.
        weight_decay (float): The L2 regularization strength.
        optimizer (str): The optimizer to use.

    Returns:
        torch.optim.Adam: The optimizer for the model.
    '''

    # YOUR CODE HERE
    # Adam optimiser with L2 regularisation
    if optimizer == "adam":
        opt = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(0.9, 0.999))

    # additional optimizers for hyperparameter tuning
    elif optimizer == "sgd":
        opt = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4, nesterov=True)
    elif optimizer == "rmsprop":
        opt = torch.optim.RMSprop(model.parameters(), lr=1e-3, alpha=0.99, eps=1e-8, weight_decay=1e-5)
    else:
        raise ValueError("Invalid optimizer. Please choose 'adam', 'sgd', or 'rmsprop'")

    return opt

<a id="section-6"></a>
### **Section 6: Training the Networks (5 points)**

In this section, you will complete the `train` function and use it to train both the `TwoLayerNet` and `ConvNet` models. You will use the custom `CIFAR100_loader`, transform function, and optimizer function that you implemented. The goal is to compare the performance of the two models on the CIFAR-100 dataset.

In [ ]:
def validate(net, testloader):
    '''
    Validates the model on the test dataset.

    Args:
        net (torch.nn.Module): The neural network model.
        testloader (torch.utils.data.DataLoader): The data loader for the test dataset.

    Returns:
        test_results (dict): Dictionary with the accuracy, precision, recall, and f1 scores for the model.
    '''

    # Set the model to evaluation mode
    net.eval()

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = net.to(device)

    correct, total = 0, 0
    y_true, y_pred = [], []
    test_results = {}

    # Disable gradient computation
    with torch.no_grad():

        # Iterate over the test dataset
        for inputs, labels in testloader:

            # move data to device
            inputs, labels = inputs.to(device), labels.to(device)

            # pass inputs to model
            outputs = net(inputs)

            # get predicted class
            _, predicted = torch.max(outputs, 1)

            # sum correct predictions & total predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_true += labels.tolist()
            y_pred += predicted.tolist()

    # calculate metrics
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred, average='weighted', zero_division=0.0)
    recall = metrics.recall_score(y_true, y_pred, average='weighted')
    f1 = metrics.f1_score(y_true, y_pred, average='weighted')

    test_results['accuracy'] = accuracy
    test_results['precision'] = precision
    test_results['recall'] = recall
    test_results['f1'] = f1

    return test_results

In [ ]:
def validate_per_class(net, testloader, classes):
    '''
    Validates the model on the test dataset per class.

    Args:
        net (torch.nn.Module): The neural network model.
        testloader (torch.utils.data.DataLoader): The data loader for the test dataset.
        classes (tuple): The tuple of class names.

    Returns:
        class_accuracy (dictionary): The accuracy per class for the model.
    '''

    # Set the model to evaluation mode
    net.eval()

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = net.to(device)

    class_correct = [0. for _ in range(len(classes))]
    class_total = [0. for _ in range(len(classes))]

    class_accuracy = {}

    # Disable gradient computation
    with torch.no_grad():

        # Iterate over the test dataset
        for inputs, labels in tqdm(testloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_predictions = (predicted == labels).squeeze()

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += correct_predictions[i].item()
                class_total[label] += 1

    for i, class_name in enumerate(classes):
        accuracy = 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
        class_accuracy[class_name] = accuracy

    return class_accuracy

In [ ]:
def train(net, train_loader, test_loader, criterion, optimizer, epochs=100):
    '''
    Trains the neural network model.

    Args:
        net (torch.nn.Module): The neural network model.
        train_loader (torch.utils.data.DataLoader): The data loader for the training dataset.
        test_loader (torch.utils.data.DataLoader): The data loader for the test dataset.
        criterion (torch.nn.Module): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer for the model.
        epochs (int): The number of epochs to train the model.

    Returns:
        epoch_performances (list): Performance of the model for each epoch. For plotting.
        epoch_losses (list): Loss for each epoch. For plotting.
    '''

    # YOUR CODE HERE
    # Set the model to train mode
    net.train()

    # Determine the device to run the model on
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = net.to(device)

    epoch_performances = []
    epoch_performances_train = []
    epoch_losses = []

    for epoch in tqdm(range(epochs)):
        epoch_loss = 0.0
        # Iterate over the train dataset
        for inputs, labels in train_loader:
            
            optimizer.zero_grad()
            inputs, labels = inputs.to(device), labels.to(device).long()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= len(train_loader) # average over the batch
        epoch_losses.append(epoch_loss)

        epoch_performances.append(validate(net, test_loader))
        epoch_performances_train.append(validate(net, train_loader))

        if type(train_loader)==list:
            random.shuffle(train_loader)

    return epoch_performances, epoch_losses


In [ ]:
def plot_epoch_performances(epoch_performances, model_name, metric="accuracy"):
    '''
    Plots the performance of the model for each epoch.

    Args:
        epoch_performances (list): List of dictionaries of the performance of the model for each epoch (accuracy, precision, recall, f1).
        metric (str): The metric to plot.
    '''
    print("metric: ", metric)
    # list of values from 0 to number of epochs
    epoch = [i for i in range(len(epoch_performances))]
    print("epoch length: ", len(epoch_performances))

    if metric == "loss":
      print("hello world")
      y = epoch_performances

    else:
      # get list of metric values for each epoch from ditionary
      y = [epoch_performances[i][metric] for i in range(len(epoch_performances))]

    plt.title(f"Test {metric} over training epochs for model {model_name}")
    plt.xlabel("Epoch")
    plt.ylabel(f"{metric}")
    plt.plot(epoch, y)
    plt.show()

First, initialize the datasets and data loaders for both models.

In [ ]:
# YOUR CODE HERE

def create_dataloaders(additional_transform=True, batch_size=512, use_cache=False, dataset='CIFAR100'):
    '''
    Create the data loaders for the CIFAR 100 dataset

    Args:
      additional_transform (bool): whether to use additional transformations
      batch_size (int): the batch size for the data loaders
      use_cache (bool): whether to use the cache for speeding up process

    Returns:
        train_data_loader (torch.utils.data.DataLoader): Data loader for the training dataset
        test_data_loader (torch.utils.data.DataLoader): Data loader for the test dataset
    '''


    # Define the transformations
    transform = create_transforms(additional_transform=additional_transform)

    # Load the training set
    if dataset == 'CIFAR100':
        train_set = CIFAR100_loader(root='./data', train=True, download=True, transform=transform)
    elif dataset == 'STL10':
        train_set = STL10_loader(root='./data', train=True, download=True, transform=transform)
    else:
        print('Invalid dataset choice')

    # Load the test set
    if dataset == 'CIFAR100':
        test_set = CIFAR100_loader(root='./data', train=False, download=True, transform=transform)
    elif dataset == 'STL10':
        test_set = STL10_loader(root='./data', train=False, download=True, transform=transform)
    else:
        print('Invalid dataset choice')

    # Create data loaders for the entire CIFAR-100 dataset
    train_data_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers = 2, persistent_workers=True)
    test_data_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size, num_workers = 2, persistent_workers=True)

    # for speeding up the data loading
    if use_cache:
      cache_train = []
      print ("warming up train loader")
      for idx, data in enumerate(tqdm(train_data_loader)):
          cache_train.append(data)

      cache_test = []
      print ("warming up test loader")
      for idx, data in enumerate(tqdm(test_data_loader)):
          cache_test.append(data)
      return cache_train, cache_test

    else:
      return train_data_loader, test_data_loader



Next, train the TwoLayerNet model on the CIFAR-100 dataset using the training data loader.

In [ ]:
train_data_loader, test_data_loader = create_dataloaders(use_cache=True)

In [ ]:
# YOUR CODE HERE
epochs = 100
learning_rate=0.001

mlp = TwoLayerNet(32 * 32 * 3, 1024, 100)
criterion = nn.CrossEntropyLoss()
optimizer_mlp = create_optimizer(mlp, learning_rate)

mlp_epoch_performances, mlp_epoch_losses = train(mlp, train_data_loader, test_data_loader, criterion, optimizer_mlp, epochs)
plot_epoch_performances(mlp_epoch_performances, model_name="MLP")
plot_epoch_performances(mlp_epoch_losses, model_name="MLP", metric="loss")

test_mlp = validate(mlp, test_data_loader)
print("MLP test accuracy: ", test_mlp)

test_mlp_per_class = validate_per_class(mlp, test_data_loader, classes)
print("Test Accuracy MLP per class: ", test_mlp_per_class)

Finally, train the ConvNet model on the CIFAR-100 dataset using the training data loader.

In [ ]:
# YOUR CODE HERE
epochs = 100
learning_rate=0.001

conv_net = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer_cnn = create_optimizer(conv_net, learning_rate)

cnn_epoch_performances, cnn_epoch_losses = train(conv_net, train_data_loader, test_data_loader, criterion, optimizer_cnn, epochs)
plot_epoch_performances(cnn_epoch_performances, model_name="CNN")
plot_epoch_performances(cnn_epoch_losses, model_name="CNN", metric="loss")

test_accuracy_cnn = validate(conv_net, test_data_loader)
test_accuracy_cnn_per_class = validate_per_class(conv_net, test_data_loader, classes)

print("CNN test accuracy: ", test_accuracy_cnn)
print("Test Accuracy CNN per class: ", test_accuracy_cnn_per_class)

<a id="section-7"></a>
### **Section 7: Setting Up the Hyperparameters (14 points)**

In this section, you will experiment with both the `ConvNet` and `TwoLayerNet` models by setting up and tuning the hyperparameters to achieve the highest possible accuracy. You have the flexibility to modify the training process, including the `train` function, `DataLoader`, `transform` functions, and optimizer as needed.

1. Adjust the hyperparameters, including learning rate, batch size, number of epochs, optimizer, weight decay, and transform function to improve the performance of both networks. Modify the training procedure and architecture as necessary. You can also add components like Batch Normalization layers.
2. Add two more layers to both `TwoLayerNet` and `ConvNet`. You can decide the size and placement of these layers. Evaluate if these changes result in higher performance and explain your findings.
3. Show the final results and describe the modifications made to enhance performance. Discuss the impact of hyperparameter tuning on both `TwoLayerNet` and `ConvNet`.
4. Compare the two networks in terms of architecture, performance, and learning rates. Provide a detailed explanation of the differences observed.

**Note:** Do not use external pre-trained networks and limit additional convolutional layers to a maximum of three beyond the original architecture.

Test the performance of TwoLayerNet after hyperparameter tuning and compare it with the ConvNet model. Provide a detailed explanation of the results.

In [ ]:
def sweep(train_data_loader, test_data_loader, learning_rate=0.001, batch_size=512, epochs=100, optimizer='adam', weight_decay=1.0, nonlinear_fn=None, transformations=False, model='cnn', chkpt=None, extra_layers=False):
    '''
    Sweeps hyperparameters for hyperparameter tuning

    Args:
        train_data_loader (torch.utils.data.DataLoader): The data loader for the training dataset.
        test_data_loader (torch.utils.data.DataLoader): The data loader for the test dataset.
        learning_rate (float): The learning rate to test.
        batch_size (int): The batch size to test.
        epochs (int): The number of epochs to test.
        optimizer (str): The optimizer to test.
        weight_decay (float): The weight decay to test.
        nonlinear_fn (str): The non-linear function to test for the CNN model.
        transformations (bool): The data transformations to test.
        model (str): The model (CNN or MLP).
        chkpt (torch.nn.Module): Checkpoint of the model if it already exists for fast loading.
        extra_layers (bool): For testing architecture of the model with additional layers.

    Returns:
        results (dict): A dict of 2 dictionaries containing the test metrics and accuracy per class
        model (torch.nn.Module): The model to save.
    '''

    # original hyperparameters
    # lr = 0.001, batch_size = 512, epochs = 100, optimizer = AdamW, weight_decay = 1, nonlinear_fn = sigmoid/tanh, layers = original, transformations = original

    # if the model checkpoint doesn't exist instantiate the network & train as usual
    if chkpt is None:
        if model == "mlp":
            net = TwoLayerNet(input_size=32 * 32 * 3, hidden_size=1024, num_classes=100, extra_layers=extra_layers)
        elif model == "cnn":
            net = ConvNet(non_linear=nonlinear_fn, extra_layers=extra_layers)
        else:
            raise ValueError("Invalid model. Please choose 'mlp' or 'cnn'")

        criterion = nn.CrossEntropyLoss()
        opt = create_optimizer(net, learning_rate=learning_rate, weight_decay=weight_decay, optimizer=optimizer)
        train(net=net, train_loader=train_data_loader, test_loader=test_data_loader, criterion=criterion, optimizer=opt, epochs=epochs)

    # if model checkpoint exists, use checkpoing
    elif chkpt is not None:
        net = chkpt

    else:
        raise ValueError("Invalid checkpoint. Please provide a valid checkpoint.")

    # validate test accuracy for the model & per class
    test_accuracy = validate(net, test_data_loader)
    test_accuracy_per_class = validate_per_class(net, test_data_loader, classes)

    results = {"metrics": test_accuracy, "class accuracies": test_accuracy_per_class}

    return results, net

    # YOUR CODE HERE

In [ ]:
def read_sweep_results(results_file):

    """
    Read the sweep results from the JSON file in case of a crash or for easy loading.

    Args:
        results_file (str): Path to the JSON file containing the results.

    Returns:
        dict: A dictionary containing the hyperparameter sweep results.

    """

    if os.path.exists(results_file):
        with open(results_file, "r") as f:
            return json.load(f)
    return {}

In [ ]:
def run_sweep(train_data_loader, test_data_loader, hyperparameter_name, hyperparameter_values, sweep_results, m, save_path="model_chkpts", results_file="sweep_results.json"):
    """
    Run sweeps for different hyperparameters, save model checkpoints, and write results to disk.

    Args:
        train_data_loader (torch.utils.data.DataLoader): The data loader for the training dataset.
        test_data_loader (torch.utils.data.DataLoader): The data loader for the test dataset.
        hyperparameter_name (str): Name of the hyperparameter to be tested (e.g. 'learning_rate', 'batch_size').
        hyperparameter_values (list): A list of test values for the hyperparameter.
        sweep_results (dict): Global dictionary for storing sweep results.
        m (str): Model name ('mlp' or 'cnn').
        save_path (str): Directory path where model checkpoints will be saved.
        results_file (str): Path to the JSON file where sweep results will be saved.

    Returns:
        None
    """

    # make sure the save path directory actually exists
    os.makedirs(save_path, exist_ok=True)

    if hyperparameter_name != "extra_layers":
        extra_layer = False
    elif hyperparameter_name == "extra_layers":
        extra_layer = hyperparameter_values[0]

    for value in hyperparameter_values:
        print("Running sweep for: ", hyperparameter_name, value)

        # Define the model filename based on hyperparameter value
        model_filename = f"{m}_model_{hyperparameter_name}_{value}.pt"
        model_filepath = os.path.join(save_path, model_filename)

        # Check if the model checkpoint already exists
        if os.path.exists(model_filepath):
            print(f"Loading existing model from: {model_filepath}")

            # instantiate a network
            if m == "cnn":
                model = ConvNet(extra_layers=extra_layer)
            elif m == "mlp":
                model = TwoLayerNet(32 * 32 * 3, 1024, 100, extra_layers=extra_layer)
            else:
                raise ValueError("Invalid model. Please choose 'mlp' or 'cnn'")

            # load the checkpoint weights onto the model
            model.load_state_dict(torch.load(model_filepath, weights_only=True))

            # pass hyperparameter testing values to the sweep function
            kwargs = {'train_data_loader': train_data_loader, 'test_data_loader': test_data_loader, hyperparameter_name: value, 'model': m, 'chkpt': model}
            results, model = sweep(**kwargs)

            if hyperparameter_name not in sweep_results:
                sweep_results[hyperparameter_name] = {}

            sweep_results[hyperparameter_name][value] = results

            print("Results retrieved for: ", hyperparameter_name, value)

        else:
            # run the sweep for the current hyperparameter value
            # ex. sweep(learning_rate=0.01)
            kwargs = {'train_data_loader': train_data_loader, 'test_data_loader': test_data_loader, hyperparameter_name: value, 'model': m}
            results, model = sweep(**kwargs)

            # store the results for the current hyperparameter value
            if hyperparameter_name not in sweep_results:
                sweep_results[hyperparameter_name] = {}
            sweep_results[hyperparameter_name][value] = results

            # save the model checkpoint in case of a crash
            torch.save(model.state_dict(), model_filepath)

            # save the sweep results to a JSON file in case of a crash
            with open(f"{m}_{results_file}", "w") as f:
                json.dump(sweep_results, f)

            print("Results saved for: ", hyperparameter_name, value)

Test the performance of ConvNet after hyperparameter tuning and compare it with the TwoLayerNet model. Provide a detailed explanation of the results.

In [ ]:
cnn_sweep_results = {'original': {}, 'learning_rate': {}, 'batch_size': {}, 'epochs': {}, 'optimizer': {}, 'weight_decay': {}, 'nonlinear_fn': {}, 'layers': {}, 'transformations': {}}
mlp_sweep_results = {'original': {}, 'learning_rate': {}, 'batch_size': {}, 'epochs': {}, 'optimizer': {}, 'weight_decay': {}, 'nonlinear_fn': {}, 'layers': {}, 'transformations': {}}

In [ ]:
torch.cuda.empty_cache()
# create train & test data loaders once
train_data_loader, test_data_loader = create_dataloaders(use_cache=True)

In [ ]:
# mlp baseline vals with original hyperparams
results, model = sweep(train_data_loader, test_data_loader)
os.makedirs("model_chkpts/", exist_ok=True)
torch.save(model.state_dict(), f"model_chkpts/mlp_model_original.pt")
mlp_sweep_results['original'] = results

with open("mlp_sweep_results.json", "w") as f:
    json.dump(mlp_sweep_results, f)

In [ ]:
# cnn baseline vals with original hyperparams
results, model = sweep(train_data_loader, test_data_loader)
torch.save(model.state_dict(), f"model_chkpts/cnn_model_original.pt")
cnn_sweep_results['original'] = results

with open("cnn_sweep_results.json", "w") as f:
    json.dump(cnn_sweep_results, f)


In [ ]:
# lists of hyperparameters for tuning
cnn_hyperparam_names = ["learning_rate", "batch_size", "epochs", "optimizer", "weight_decay", "nonlinear_fn", "transformations", "extra_layers"]
mlp_hyperparam_names = ["learning_rate", "batch_size", "epochs", "optimizer", "weight_decay", "transformations", "extra_layers"]

learning_rates = [0.0001, 0.01]
batch_sizes = [32, 64, 128, 256, 1024]
epochs = [10, 50, 200, 500]
optimizers = ['sgd', 'rmsprop']
weight_decays = [0.8, 0.6, 0.4, 0.2, 0.1, 0.01]
nonlinear_fns = ["relu"]
transformations = [True]
layers = [True]

cnn_hyperparams = [learning_rates, batch_sizes, epochs, optimizers, weight_decays, nonlinear_fns, transformations, layers]
mlp_hyperparams = [learning_rates, batch_sizes, epochs, optimizers, weight_decays, transformations, layers]

In [ ]:
# Tune MLP
m = "mlp"

# Run sweeps
for i in tqdm(range(len(mlp_hyperparam_names))):
    run_sweep(train_data_loader, test_data_loader, mlp_hyperparam_names[i], mlp_hyperparams[i], mlp_sweep_results, m)

In [ ]:
# Tune CNN
m = "cnn"

# Run sweeps
for i in tqdm(range(len(cnn_hyperparam_names))):
    run_sweep(train_data_loader, test_data_loader, cnn_hyperparam_names[i], cnn_hyperparams[i], cnn_sweep_results, m)

In [ ]:
def json_to_dict(json_file):
    """
    Loads a JSON file to dictionary (to use in case of crash or sweep dictionary is lost from memory)

    Args:
        json_file (str): Path to the JSON file.

    Returns:
        dict: The contents of the JSON file as a dictionary.
    """
    with open(json_file, 'r') as f:
        data = json.load(f)

    return data

cnn_sweep_results = json_to_dict("cnn_sweep_results.json")
mlp_sweep_results = json_to_dict("mlp_sweep_results.json")
print(cnn_sweep_results)
print(mlp_sweep_results)

In [ ]:
# Analyze CNN tuning

def plot_hyperparameter_comparison(sweep_results, model, metric="accuracy"):
    """
    Plot the hyperparameter tuning vs baseline metrics

    Args:
        sweep_results (dict): Dictionary containing the sweep results.
        model (str): The model name (e.g., "mlp", "cnn").
        metric (str): The metric to compare (e.g., "accuracy", "precision", etc.).

    Returns:
        None
    """
    # Extract baseline metrics
    baseline_metrics = sweep_results["original"]["metrics"]
    baseline_value = baseline_metrics.get(metric, None)

    # Prepare data for plotting
    hyperparameters = []
    values = []
    baseline = []

    for hyperparam, results in sweep_results.items():
        # skip baseline
        if hyperparam == "original":
            continue

        for param_value, result in results.items():
            hyperparameters.append(f"{hyperparam}={param_value}")
            values.append(result["metrics"].get(metric, None))
            baseline.append(baseline_value)

    # Plot
    plt.figure(figsize=(10, 6))

    plt.plot(hyperparameters, values, label="Tuned results", marker="o", color="blue")
    plt.plot(hyperparameters, baseline, label="Baseline", linestyle="--", marker="x", color="red")

    plt.xticks(rotation=45, ha="right")
    plt.xlabel("Hyperparameters")
    plt.ylabel(metric.capitalize())
    plt.title(f"Comparison of {metric.capitalize()} across Hyperparameters for {model}")
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage
plot_hyperparameter_comparison(cnn_sweep_results, model="cnn", metric="accuracy")
# plot_hyperparameter_comparison(cnn_sweep_results, model="cnn", metric="precision")
# plot_hyperparameter_comparison(cnn_sweep_results, model="cnn", metric="recall")
# plot_hyperparameter_comparison(cnn_sweep_results, model="cnn", metric="f1")

plot_hyperparameter_comparison(mlp_sweep_results, model="mlp", metric="accuracy")
# plot_hyperparameter_comparison(mlp_sweep_results, model="mlp", metric="precision")
# plot_hyperparameter_comparison(mlp_sweep_results, model="mlp", metric="recall")
# plot_hyperparameter_comparison(mlp_sweep_results, model="mlp", metric="f1")

Add two more layers to both `TwoLayerNet` and `ConvNet`. You can decide the size and placement of these layers. Evaluate if these changes result in higher performance and explain your findings.

In [ ]:
# Tuned model MLP performance
epochs = 10
learning_rate = 0.0001
batch_size = 64
extra_layers = True
transformations = True
weight_decay = 0.8

train_data_loader_mlp, test_data_loader_mlp = create_dataloaders(additional_transform=transformations, batch_size=batch_size, use_cache=True)

In [ ]:
mlp = TwoLayerNet(32 * 32 * 3, 1024, 100, extra_layers=extra_layers)
criterion = nn.CrossEntropyLoss()
optimizer_mlp = create_optimizer(mlp, learning_rate, weight_decay)

mlp_epoch_performances, mlp_epoch_losses = train(mlp, train_data_loader_mlp, test_data_loader_mlp, criterion, optimizer_mlp, epochs)
plot_epoch_performances(mlp_epoch_performances, model_name="MLP")
plot_epoch_performances(mlp_epoch_losses, model_name="MLP", metric="loss")

test_mlp = validate(mlp, test_data_loader_mlp)
print("MLP test accuracy: ", test_mlp)

test_mlp_per_class = validate_per_class(mlp, test_data_loader_mlp, classes)
print("Test Accuracy MLP per class: ", test_mlp_per_class)

In [ ]:
# Tuned model CNN performance
epochs = 100
learning_rate = 1e-3
batch_size = 512
extra_layers = True
transformations = True
weight_decay = 0.8
nonlinear_fn = "relu"

train_data_loader_cnn, test_data_loader_cnn = create_dataloaders(additional_transform=transformations, batch_size=batch_size, use_cache=True)


In [ ]:
conv_net = ConvNet(extra_layers=extra_layers)
criterion = nn.CrossEntropyLoss()
optimizer_cnn = create_optimizer(conv_net, learning_rate, weight_decay)

cnn_epoch_performances, cnn_epoch_losses = train(conv_net, train_data_loader_cnn, test_data_loader_cnn, criterion, optimizer_cnn, epochs)
plot_epoch_performances(cnn_epoch_performances, model_name="CNN")
plot_epoch_performances(cnn_epoch_losses, model_name="CNN", metric="loss")

test_accuracy_cnn = validate(conv_net, test_data_loader_cnn)
print("CNN test accuracy: ", test_accuracy_cnn)

test_accuracy_cnn_per_class = validate_per_class(conv_net, test_data_loader_cnn, classes)
print("Test Accuracy CNN per class: ", test_accuracy_cnn_per_class)


In [ ]:
def plot_accuracy_per_class(model_name, accuracy_per_class):
  """
    Plot the top 5 and bottom 5 accuracies

    Args:
        model_name (str): Name of the model.
        accuracy_per_class (dict): Dictionary containing class names and accuracies.

    Returns:
        None
  """
  # Extract class names and accuracies
  sorted_accuracy = sorted(accuracy_per_class.items(), key=lambda x: x[1], reverse=True)

  # Get the bottom 5 and top 5 accuracies
  bottom_5 = sorted_accuracy[-5:]
  top_5 = sorted_accuracy[:5]

  # Combine them for plotting
  combined = top_5 + bottom_5
  classes, accuracies = zip(*combined)  # Unzip into two lists

  # Plotting
  plt.figure(figsize=(10, 5))
  plt.bar(classes, accuracies, color=['red' if acc < 30 else 'green' for acc in accuracies])

  for i in range(len(combined)):
        plt.text(i, accuracies[i], accuracies[i], ha = 'center')

  plt.title(f'Top 5 and Bottom 5 Class Accuracies for {model_name}')
  plt.xlabel('Class')
  plt.ylabel('Accuracy (%)')
  plt.xticks(rotation=45)
  plt.ylim(0, 100)
  plt.grid(axis='y')

  # Show the plot
  plt.show()

In [ ]:
plot_accuracy_per_class("MLP", test_mlp_per_class)
plot_accuracy_per_class("CNN", test_accuracy_cnn_per_class)

In [ ]:
class ConvNetFinal(nn.Module):
    def __init__(self):
        super(ConvNetFinal, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.fcon1 = nn.Linear(in_features=512, out_features=256)
        self.fcon2 = nn.Linear(in_features=256, out_features=128)
        self.fcon3 = nn.Linear(in_features=128, out_features=100)
        self.drop = nn.Dropout(p=0.55)
        self.maxp1 = nn.MaxPool2d(kernel_size=2)
        self.bn64 = nn.BatchNorm2d(64)
        self.bn96 = nn.BatchNorm2d(96)
        self.bn128 = nn.BatchNorm2d(128)

    def forward(self, x):

        x = F.relu(self.bn64(self.conv1(x)))
        x = self.maxp1(x)

        x = F.relu(self.bn96(self.conv2(x)))
        x = self.maxp1(x)

        x = F.relu(self.bn128(self.conv3(x)))

        x = F.relu(self.bn128(self.conv4(x)))

        x = x.flatten(start_dim=1)
        x = F.relu(self.fcon1(x))
        x = self.drop(x)

        x = F.relu(self.fcon2(x))
        x = self.drop(x)

        x = self.fcon3(x)

        return x

def create_transforms(test=False):
    if test:
        tr_compose = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
        ])
    else :
        tr_compose = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5), 
            transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
        ])
    return tr_compose


conv3 = ConvNetFinal()
checkpoint = torch.load('./models/CONVNET2_MIDLRDECAY_B64.pth')
''' 
optimizer:  torch.optim.SGD(conv3.parameters(), lr=0.003, momentum=0.9, weight_decay=0.001)
LR decay:   torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
epochs:     50
batch size: 64
'''
conv3.load_state_dict(checkpoint['model_state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv3.to(device)

trainset = CIFAR100_loader(root='./data', train=True, download=False, transform=create_transforms(test=False))
testset = CIFAR100_loader(root='./data', train=False, download=False, transform=create_transforms(test=True))
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

print(validate(conv3, testloader))
validate_per_class(conv3, testloader, classes)

In [ ]:
import pandas as pd

def plot_epoch_performancesCSV(csv_path, model_name, metric="test_acc"): # metric can also be 'train_acc' or 'loss'
    '''
    Plots the performance of the model for each epoch.

    Args:
        epoch_performances (list): List of dictionaries of the performance of the model for each epoch (accuracy, precision, recall, f1).
        metric (str): The metric to plot.
    '''

    y = pd.read_csv(csv_path)[metric].to_numpy()
    plt.title(f"Test {metric} over training epochs for model {model_name}")
    plt.xlabel("Epoch")
    plt.ylabel(f"{metric}")
    plt.plot(y)
    plt.show()

plot_epoch_performancesCSV('./models/CONVNET2_MIDLRDECAY_B64.csv', 'ConvNetFinal', 'test_acc')

In [ ]:
def trainNEW(net, train_loader, test_loader, validate_loader, criterion, optimizer, epochs=100, lr_decay=True, save_model=False, savename="XYZ"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    net.train()
    best_test_acc = 0
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    val_acc = 0

    for epoch in range(epochs):
        epoch_loss = 0.0

        for i, (img, label) in enumerate(train_loader):
            img, label = img.to(device), label.to(device)

            
            optimizer.zero_grad()           # reset gradients
            out = net(img)                  # forward pass
            loss = criterion(out, label)    # calculate loss
            loss.backward()                 # backward pass
            optimizer.step()                # optimize

            epoch_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

        test_acc = validate(net, test_loader)['accuracy']
        print(f'Accuracy on test set: {test_acc}')

        if validate_loader != None:
            val_acc = validate(net, validate_loader, validate=True)

        if save_model:
            with open(f'{savename}.csv', 'a') as out:
                out.write(f'{epoch+1},{epoch_loss},{test_acc},{val_acc}\n')

        epoch_loss = 0.0

        if lr_decay:
            scheduler.step()

        if save_model:
            if test_acc > best_test_acc:
                model_path = f'./models/{savename}.pth'
                torch.save({
                    'epoch': epoch+1,
                    'model_state_dict': net.state_dict(),
                    'loss': epoch_loss,
                    'val_acc': test_acc
                }, model_path)
                best_test_acc = test_acc
    print('Finished Training')
    return summary

In [ ]:
conv3NEW = ConvNet(non_linear='relu', extra_layers=True, num_classes=100)
optimConv3N = torch.optim.SGD(conv3NEW.parameters(), lr=0.003, momentum=0.9, weight_decay=0.001)

SAVENAME = "XYZ"
print(f"training: {SAVENAME}")
with open(f'{SAVENAME}.csv', 'w') as out:
    out.write('epoch,loss,test_acc,train_acc\n')
trainNEW(conv3NEW, trainloader, testloader, None, nn.CrossEntropyLoss(), optimConv3N, epochs=50, save_model=True)

<a id="section-8"></a>
### **Section 8: Visualizing the STL-10 Dataset and Preparing the Data Loader (3 points)**

In this section, you will work with a subset of the [STL-10](https://cs.stanford.edu/~acoates/stl10/) dataset, containing higher resolution images and different object classes than CIFAR-100. Before fine-tuning your ConvNet on this dataset, first complete the `visualise_stl10` function to display sample images from the following 5 classes:

1. **Bird**
2. **Deer**
3. **Dog**
4. **Horse**
5. **Monkey**

In [ ]:
def visualise_stl10(class_mapping, train_set):
    '''
    Visualizes 5 images from each specified class in the STL-10 dataset.

    Args:
        class_mapping (dict): A dictionary mapping class indices to class names.
    '''

    # YOUR CODE HERE
    data = torch.tensor(train_set.data)
    labels = torch.tensor(train_set.labels)
    fig, axs = plt.subplots(num_classes_per_superclass, num_samples, figsize=(25, 20))

    for index, label in enumerate(class_mapping.keys()):
        subset = data[labels == label]
        random_indices = torch.randperm(subset.size(0))
        
        # Set the class name as the label for the first column
        class_name = class_mapping[label]
        axs[index, 0].set_ylabel(class_name, fontsize=16, rotation=0, labelpad=80, va='center')
        
        for num_sample, sample_index in enumerate(random_indices[ : num_samples]):
            axs[index, num_sample].imshow(subset[sample_index].permute(1, 2, 0))
            axs[index, num_sample].set_xticks([])
            axs[index, num_sample].set_yticks([])
        

In [ ]:
# Define the transformations

transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Load the STL-10 training set
#train_set = torchvision.datasets.STL10(root='./data', download=True, transform=transform)

# Define the class mapping for bird, deer, dog, horse, and monkey
class_mapping = {1: 'bird', 4: 'deer', 5: 'dog', 6: 'horse', 7: 'monkey'}

# Visualize STL-10 classes
visualise_stl10(class_mapping, train_set)

After visualizing the data, implement the `STL10_loader` class to create a custom data loader that initializes the dataset, extracts the target classes, and applies the necessary image transformations. Once these tasks are completed, you will move on to fine-tuning the ConvNet on this dataset in the next section.

In [ ]:
class STL10_loader(Dataset):
    def __init__(self, root, train=True, transform=None, download=False):
        '''
        Initializes the STL10 dataset.

        Args:
            root (str): Root directory of the dataset.
            train (bool): If True, use the training set, otherwise use the test set.
            transform (callable, optional): A function/transform to apply to the images.
        '''

        # YOUR CODE HERE
        self.dataset = torchvision.datasets.STL10(root=root, split=('train' if train else 'test'), download=False, transform=transform)
        self.labels = torch.tensor(self.dataset.labels)
        target_classes = torch.tensor([1, 4, 5, 6, 7])

        # Filter data and labels by target classes
        self.filtered_indices = torch.isin(self.labels, target_classes)
        self.filtered_data = torch.tensor(self.dataset.data[self.filtered_indices])
        self.filtered_labels = self.labels[self.filtered_indices]


    def __len__(self):
        '''
        Returns the number of samples in the dataset.
        '''

        # YOUR CODE HERE
        return len(self.filtered_data)

    def __getitem__(self, idx):
        '''
        Retrieves a sample from the dataset at the specified index.

        Args:
            idx (int): The index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the transformed image and its target label.
        '''

        # YOUR CODE HERE
        image = self.filtered_data[idx]
        label = self.filtered_labels[idx]

        return image.float(), label.long()

In [ ]:
# from pathlib import Path

# class STL10_loader(Dataset):
#     def __init__(self, root, train=True, transform=None):
#         '''
#         Initializes the STL10 dataset.

#         Args:
#             root (str): Root directory of the dataset.
#             train (bool): If True, use the training set, otherwise use the test set.
#             transform (callable, optional): A function/transform to apply to the images.
#         '''

#         # YOUR CODE HERE
#         self.root = root
#         self.train = train
#         self.transform = transform
#         self.images = []
#         self.labels = []

#         CWD = Path(os.getcwd())
#         ROOT = CWD/Path(self.root)
        
#         with open(ROOT/Path(f"stl10_binary/{"train" if train else "test"}_X.bin"), "rb") as f:
#             num_imgs = 5000 if train else 8000
#             imgs = np.fromfile(f, dtype=np.uint8)
#             imgs = imgs.reshape((num_imgs, 3, 96, 96))
#             imgs = np.transpose(imgs, (0, 2, 3, 1))
#             for i in range(num_imgs):
#                 self.images.append(Image.fromarray(imgs[i,:,:,:]))

#         with open(ROOT/Path(f"stl10_binary/{"train" if train else "test"}_y.bin"), "rb") as f:
#             self.labels = list(np.fromfile(f, dtype=np.uint8) - 1)

#         labels_to_keep = [1,4,5,6,7]
#         self.images = [img for img,lab in zip(self.images, self.labels) if lab in set(labels_to_keep)]
#         self.labels = [lab for lab in self.labels if lab in set(labels_to_keep)]
#         for idx in range(len(self.labels)):
#             for i,j in enumerate(labels_to_keep):
#                 self.labels[idx] = i if (self.labels[idx] == j) else self.labels[idx]
            

        
#     def __len__(self):
#         '''
#         Returns the number of samples in the dataset.
#         '''

#         # YOUR CODE HERE
#         return len(self.labels)

#     def __getitem__(self, idx):
#         '''
#         Retrieves a sample from the dataset at the specified index.

#         Args:
#             idx (int): The index of the sample to retrieve.

#         Returns:
#             tuple: A tuple containing the transformed image and its target label.
#         '''

#         # YOUR CODE HERE
#         if self.transform:
#             return self.transform(self.images[idx]), self.labels[idx]
#         else:
#             to_tensor = transforms.ToTensor()
#             return to_tensor(self.images[idx]), self.labels[idx]

<a id="section-9"></a>
### **Section 9: Fine-tuning ConvNet on STL-10 (14 points)**

In this section, you will load the pre-trained parameters of the ConvNet (trained on CIFAR-100) and modify the output layer to adapt it to the new dataset containing 5 classes. You can either first load the pre-trained parameters and then modify the output layer, or change the output layer before loading the matched pre-trained parameters. Once modified, you will train the model and document the settings of hyperparameters, accuracy, and learning curve. Additionally, visualize both the training loss and accuracy to assess the learning process. To gain a deeper understanding of the feature learning process, consider using techniques like [**t-sne**](https://lvdmaaten.github.io/tsne/) for feature space visualization.

In [ ]:
# YOUR CODE HERE
# Tuned model CNN performance
epochs = 100
learning_rate = 1e-3
batch_size = 512
extra_layers = True
transformations = True
weight_decay = 0.8
nonlinear_fn = "relu"

train_data_loader_cnn, test_data_loader_cnn = create_dataloaders(additional_transform=transformations, batch_size=batch_size, use_cache=True, dataset='STL10')

filtered_state_dict = {k: v for k, v in conv_net.state_dict().items() if k not in ['model.20.weight', 'model.20.bias']}
new_conv_net.load_state_dict(filtered_state_dict, strict=False)
new_conv_net = ConvNet(extra_layers=extra_layers, num_classes=5)
new_conv_net.load_state_dict(filtered_state_dict, strict=False)

criterion = nn.CrossEntropyLoss()
optimizer_cnn = create_optimizer(new_conv_net, learning_rate, weight_decay)

cnn_epoch_performances, cnn_epoch_losses = train(new_conv_net, train_data_loader_cnn, test_data_loader_cnn, criterion, optimizer_cnn, epochs)
plot_epoch_performances(cnn_epoch_performances, model_name="CNN")
plot_epoch_performances(cnn_epoch_losses, model_name="CNN", metric="loss")

test_accuracy_cnn = validate(new_conv_net, test_data_loader_cnn)
print("CNN test accuracy: ", test_accuracy_cnn)

test_accuracy_cnn_per_class = validate_per_class(new_conv_net, test_data_loader_cnn, classes)
print("Test Accuracy CNN per class: ", test_accuracy_cnn_per_class)

In [ ]:
print(conv_net.state_dict().keys())

<a id="section-10"></a>
### **Section 10: Bonus Challenge (optional)**

Try to achieve the highest possible accuracy on the test dataset (5 classes from STL-10) by adjusting hyperparameters, modifying architectures, or applying techniques like data augmentation. The top-performing teams will earn bonus points that can significantly boost their final lab grade, even allowing it to exceed 10 (up to 11):

- **1st place:** +1.0 to the final grade of the final lab
- **2nd place:** +0.8 to the final grade of the final lab
- **3rd place:** +0.6 to the final grade of the final lab
- **4th place:** +0.4 to the final grade of the final lab
- **5th place:** +0.2 to the final grade of the final lab

**Hint:** You may use techniques like data augmentation, freezing early layers, modifying architecture, or optimizing hyperparameters. Only data from CIFAR-100 and STL-10 can be used, and you cannot add more than 3 additional convolutional layers.

In [ ]:
# YOUR CODE HERE
from pathlib import Path

class STL10_loader(Dataset):
    def __init__(self, root, train=True, transform=None):
        '''
        Initializes the STL10 dataset.

        Args:
            root (str): Root directory of the dataset.
            train (bool): If True, use the training set, otherwise use the test set.
            transform (callable, optional): A function/transform to apply to the images.
        '''

        # YOUR CODE HERE
        self.root = root
        self.train = train
        self.transform = transform
        self.images = []
        self.labels = []

        CWD = Path(os.getcwd())
        ROOT = CWD/Path(self.root)
        
        with open(ROOT/Path(f"stl10_binary/{"train" if train else "test"}_X.bin"), "rb") as f:
            num_imgs = 5000 if train else 8000
            imgs = np.fromfile(f, dtype=np.uint8)
            imgs = imgs.reshape((num_imgs, 3, 96, 96))
            imgs = np.transpose(imgs, (0, 2, 3, 1))
            imgs = np.rot90(imgs, 3, (1,2))
            for i in range(num_imgs):
                self.images.append(Image.fromarray(imgs[i,:,:,:]))

        with open(ROOT/Path(f"stl10_binary/{"train" if train else "test"}_y.bin"), "rb") as f:
            self.labels = list(np.fromfile(f, dtype=np.uint8) - 1)

        labels_to_keep = [1,4,5,6,7]
        self.images = [img for img,lab in zip(self.images, self.labels) if lab in set(labels_to_keep)]
        self.labels = [lab for lab in self.labels if lab in set(labels_to_keep)]
        for idx in range(len(self.labels)):
            for i,j in enumerate(labels_to_keep):
                self.labels[idx] = i if (self.labels[idx] == j) else self.labels[idx]
            

        
    def __len__(self):
        '''
        Returns the number of samples in the dataset.
        '''

        # YOUR CODE HERE
        return len(self.labels)

    def __getitem__(self, idx):
        '''
        Retrieves a sample from the dataset at the specified index.

        Args:
            idx (int): The index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the transformed image and its target label.
        '''

        # YOUR CODE HERE
        if self.transform:
            return self.transform(self.images[idx]), self.labels[idx]
        else:
            to_tensor = transforms.ToTensor()
            return to_tensor(self.images[idx]), self.labels[idx]
        

# YOUR CODE HERE
def create_transforms_STL5(test=False):
    if test:
        tr_compose = transforms.Compose([
            transforms.Resize(32),
            transforms.RandomAdjustSharpness(2,p=1),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    else :
        tr_compose = transforms.Compose([
            transforms.Resize(32),
            transforms.RandomAdjustSharpness(2,p=1),
            transforms.RandomHorizontalFlip(p=0.5), 
            transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0.2),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
        ])
    return tr_compose




class ConvNetFinalSTL5(nn.Module):

    def __init__(self):

        super(ConvNetFinalSTL5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.fcon1 = nn.Linear(in_features=512, out_features=256)
        self.fcon2 = nn.Linear(in_features=256, out_features=128)
        self.fcon3 = nn.Linear(in_features=128, out_features=100)
        self.fcon4 = nn.Linear(in_features=100, out_features=5)     # now for 5 classes
        self.drop = nn.Dropout(p=0.55)
        self.maxp1 = nn.MaxPool2d(kernel_size=2)
        self.bn64 = nn.BatchNorm2d(64)
        self.bn96 = nn.BatchNorm2d(96)
        self.bn128 = nn.BatchNorm2d(128)

    def forward(self, x):

        x = F.relu(self.bn64(self.conv1(x)))
        x = self.maxp1(x)

        x = F.relu(self.bn96(self.conv2(x)))
        x = self.maxp1(x)

        x = F.relu(self.bn128(self.conv3(x)))

        x = F.relu(self.bn128(self.conv4(x)))

        x = x.flatten(start_dim=1)
        x = F.relu(self.fcon1(x))
        x = self.drop(x)

        x = F.relu(self.fcon2(x))
        x = self.drop(x)

        x = F.relu(self.fcon3(x))
        x = self.fcon4(x)

        return x



train_transforms_stl5 = create_transforms_STL5(test=False)
test_transforms_stl5 = create_transforms_STL5(test=True)

train_stl5 = STL10_loader("./data", train=True, transform=train_transforms_stl5)
test_stl5 = STL10_loader("./data", train=False, transform=test_transforms_stl5)

train_loader_stl5 = DataLoader(train_stl5, batch_size=64, shuffle=True)
test_loader_stl5 = DataLoader(test_stl5, batch_size=64, shuffle=False)



In [ ]:
pretrained = torch.load('./models/CONVNET2_FASTLRDECAY.pth')['model_state_dict']
cnstl5 = ConvNetFinalSTL5()
cnstl5.state_dict().update(pretrained)
cnstl5.load_state_dict(cnstl5.state_dict())
optimSTL5 = torch.optim.SGD(cnstl5.parameters(), lr=0.003, momentum=0.9, weight_decay=0.001)

SAVENAME = "XYZ"
print(f"training: {SAVENAME}")
with open(f'{SAVENAME}.csv', 'w') as out:
    out.write('epoch,loss,test_acc,train_acc\n')

train(cnstl5, train_loader_stl5, test_loader_stl5, train_loader_stl5, nn.CrossEntropyLoss(), optimSTL5, epochs=150, lr_decay=True, save_model=True, savename=SAVENAME)

In [ ]:
model = ConvNetFinalSTL5()
checkpoint = torch.load('./models/CONVNET2_TRANSFER_B64_IMFLIP.pth')
print(f'best performance at epoch: {checkpoint['epoch']}')
model.load_state_dict(checkpoint['model_state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(validate(model, test_loader_stl5))
validate_per_class(model, test_loader_stl5, ['bird','deer','dog','horse','monkey'])

<a id="section-x"></a>
### **Section X: Individual Contribution Report *(Mandatory)***

Because we want each student to contribute fairly to the submitted work, we ask you to fill out the textcells below. Write down your contribution to each of the assignment components in percentages. Naturally, percentages for one particular component should add up to 100% (e.g. 30% - 30% - 40%). No further explanation has to be given.

| Name                   | Contribution on Research | Contribution on Programming | Contribution on Writing |
|------------------------|--------------------------|-----------------------------|-------------------------|
| Pradyut Nair           | 25 %                     | 25 %                        | 25 %                    |
| Benjamin Shaffrey      | 25 %                     | 25 %                        | 25 %                    |
| Akshay Sardjoe Missier | 25 %                     | 25 %                        | 25 %                    |
| Christina Isaicu       | 25 %                      | 25 %                         | 25 %                     |

### - End of Notebook -